In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [30]:
from pathlib import Path
import unidecode
import pandas as pd

import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
import constants

In [4]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [5]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [6]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [7]:
belge_sayisi_extractor = BelgeSayisiExtractor()

In [8]:
belge_sayisi_extractor.extract(df)

'belge_sayi' cikariliyor... -> 'belge_sayi_pred'


  0%|          | 0/385 [00:00<?, ?it/s]

In [9]:
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=True)

0.1117

In [10]:
# NaN olmayan etiketlerde %100 başarı
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=False)

1.0

In [14]:
show_false_predictions(input_df=df,
                       feature_extractor=belge_sayisi_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=True,
                       first_n=5)

,belge_sayi,belge_sayi_pred,baslik,url,kategori
917,NaN,B.07.0.GEL.0.44/4412-17-047504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",http://www.gib.gov.tr/node/89371,Genelge
918,NaN,B.07.0.GEL.0.71/7100-89 - 009461,"GELİR İDARESİ BAŞKANLIĞI, VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2",http://www.gib.gov.tr/node/89248,Genelge
919,NaN,B. 13.2.SGK.0.71.09.00/53>~? 3 0 MAYİS 2Oİ2,"SOSYAL GÜVENLİK KURUMU, GENELGE 2012-20",https://kms.kaysis.gov.tr/Home/Goster/73822,Genelge
920,NaN,NaN,YENİDEN YAPILANDIRMA HAKKINDA GENELGE,https://www.bddk.org.tr/Mevzuat/DokumanGetir/900,Genelge
921,NaN,NaN,ENGELLİ HAKLARI İZLEME VE DEĞERLENDİRME KURULU İLE İLGİLİ 2021/23 SAYILI CUMHURBAŞKANLIĞI GENELGESİ,https://www.mevzuat.gov.tr/MevzuatMetin/CumhurbaskanligiGenelgeleri/20211203-23.pdf,Genelge


### MADDE SAYISI

In [15]:
madde_sayisi_extractor = MaddeSayisiExtractor()

In [16]:
madde_sayisi_extractor.extract(df)

'madde_sayisi' cikariliyor... -> 'madde_sayisi_pred'


  0%|          | 0/1338 [00:00<?, ?it/s]

In [23]:
calculate_accuracy(input_df=df,
                   feature_extractor=madde_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 1338
Dogru Tahmin Sayisi: 1071
Yanlis Tahmin Sayisi: 267
Basari Orani: 0.8004


In [24]:
show_false_predictions(input_df=df,
                       feature_extractor=madde_sayisi_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=False,
                       first_n=5)

,madde_sayisi,madde_sayisi_pred,baslik,url,kategori
6,76,88,MİLLÎ EĞİTİM BAKANLIĞININ TEŞKİLAT VE GÖREVLERİ HAKKINDA KANUN HÜKMÜNDE KARARNAME İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2016/12/20161209-5.htm,Kanun
8,17,94,YÜKSEKÖĞRETİM KANUNU İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2018/05/20180518-1.htm,Kanun
11,70,80,VERGİ KANUNLARI İLE BAZI KANUN VE KANUN HÜKMÜNDE KARARNAMELERDE DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2019/01/20190118-1.htm,Kanun
14,35,61,GELİR VERGİSİ KANUNU İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASI HAKKINDA KANUN,http://www.resmigazete.gov.tr/eskiler/2019/07/20190719M1-1.htm,Kanun
15,19,22,"ÖDEME VE MENKUL KIYMET MUTABAKAT SİSTEMLERİ, ÖDEME HİZMETLERİ VE ELEKTRONİK PARA KURULUŞLARI HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN",https://www.resmigazete.gov.tr/eskiler/2019/11/20191122-2.htm,Kanun


### MEVZUAT NO

In [25]:
mevzuat_no_extractor = MevzuatNoExtractor()

In [26]:
mevzuat_no_extractor.extract(df)

'mevzuat_no' cikariliyor... -> 'mevzuat_no_pred'


  0%|          | 0/1723 [00:00<?, ?it/s]

In [27]:
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=False)

Veri Sayisi: 1091
Dogru Tahmin Sayisi: 968
Yanlis Tahmin Sayisi: 123
Basari Orani: 0.8873


In [28]:
show_false_predictions(input_df=df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=False,
                       first_n=5)

,mevzuat_no,mevzuat_no_pred,baslik,url,kategori
920,2018/1,NaN,YENİDEN YAPILANDIRMA HAKKINDA GENELGE,https://www.bddk.org.tr/Mevzuat/DokumanGetir/900,Genelge
921,2021/23,NaN,ENGELLİ HAKLARI İZLEME VE DEĞERLENDİRME KURULU İLE İLGİLİ 2021/23 SAYILI CUMHURBAŞKANLIĞI GENELGESİ,https://www.mevzuat.gov.tr/MevzuatMetin/CumhurbaskanligiGenelgeleri/20211203-23.pdf,Genelge
922,2006/16,16/375ek,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge
923,2005/16-358,16/358ek,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge
1068,2010/100,NaN,SOSYAL GÜVENLİK KURUMU 2010/100 SAYILI GENELGESİ (Geçici 17 nci Madde Uygulaması Hk.),http://www.sgk.gov.tr/wps/wcm/connect/41fd3a0f-9826-4098-9d27-45e0fc75a2e5/2010_100.pdf?MOD=AJPERES&CACHEID=41fd3a0f-9826-4098-9d27-45e0fc75a2e5,Genelge


In [31]:
constants.COLS_MEVZUAT_NO

['Kanun',
 'Kanun Hükmünde Kararname',
 'Genelge',
 'Cumhurbaşkanlığı Kararnamesi',
 'Tüzük',
 'Yönetmelik']

In [37]:
filtered_df = filter_dataframe(input_df=df,
                               selected_categories=["Genelge"],
                               custom_filter_array=[df.url.str.contains("kms.kaysis.gov.tr")])

calculate_accuracy(input_df=filtered_df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=False)

show_false_predictions(input_df=filtered_df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori"],
                       include_na=False,
                       first_n=5)

Veri Sayisi: 154
Dogru Tahmin Sayisi: 116
Yanlis Tahmin Sayisi: 38
Basari Orani: 0.7532


,mevzuat_no,mevzuat_no_pred,baslik,url,kategori
922,2006/16,16/375ek,"SOSYAL GÜVENLİK KURUMU, YATIRIMLARIN VE İSTİHDAMIN TEŞVİKİNE İLİŞKİN 26.06.2006 TARİHLİ 2006-16 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/106310,Genelge
923,2005/16-358,16/358ek,"SOSYAL GÜVENLİK KURUMU, İHALE KONUSU İŞLERDEN KAYNAKLANAN BORÇLARIN TAKİBİ HAKKINDA 06/12/2005 TARİHLİ 2005/16-358 EK SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/128284,Genelge
1135,2017/12,2017u2,"SOSYAL GÜVENLİK KURUMU, FATURA BEDELLERİNİN ÖDENMESİ GENELGESİNİN YÜRÜRLÜKTEN KALDIRILMASINA DAİR 2017/12 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/104268,Genelge
1139,1046330,NaN,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/105981,Genelge
1148,2016/17,kurumumuzkadroluvesozlesmeliavukatlaritarafindanhukukmusavirligineiletilen,"SOSYAL GÜVENLİK KURUMU, HUKUK SERVİSLERİNİN İHTİYAÇLARINA İLİŞKİN 2016/17 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/75464,Genelge
